In [1]:
from os.path import abspath
from pyspark.sql import SparkSession

# set default location for [spark-warehouse]
warehouse_location = abspath('spark-warehouse')

In [21]:
!pip install 

/bin/bash: ping: command not found


In [33]:
import pyping


ModuleNotFoundError: No module named 'core'

In [32]:
r = pyping.ping('35.223.175.245')

if r.ret_code == 0:
    print("Success")
else:
    print("Failed with {}".format(r.ret_code))

NameError: name 'pyping' is not defined

In [31]:
!pip install core

ERROR: Could not find a version that satisfies the requirement core (from versions: none)
ERROR: No matching distribution found for core


In [2]:
spark = (
    SparkSession
    .builder
    .appName("Delta Lake")
    .config("spark.jars.packages", "org.apache.spark:spark-avro_2.12:3.1.1,io.delta:delta-core_2.12:1.0.0,org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.1")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .config("spark.sql.debug.maxToStringFields","1000")
    .config("spark.kryoserializer.buffer.max","128m")
    .config("spark.driver.memory", "24G")
    .enableHiveSupport()
    .getOrCreate()
)
spark.sparkContext.setLogLevel("WARN")

:: loading settings :: url = jar:file:/usr/local/spark-3.1.2-bin-hadoop3.2/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
org.apache.spark#spark-avro_2.12 added as a dependency
io.delta#delta-core_2.12 added as a dependency
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-198dc5e1-249c-4ce5-a572-5b4ed5ea9286;1.0
	confs: [default]
	found org.apache.spark#spark-avro_2.12;3.1.1 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found io.delta#delta-core_2.12;1.0.0 in central
	found org.antlr#antlr4;4.7 in central
	found org.antlr#antlr4-runtime;4.7 in central
	found org.antlr#antlr-runtime;3.5.2 in central
	found org.antlr#ST4;4.0.8 in central
	found org.abego.treelayout#org.abego.treelayout.core;1.0.3 in central
	found org.glassfish#javax.json;1.0.4 in central
	found com.ibm.icu#icu4j;58.2 in central
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.1.1 in central
	found org.apache.spark#spark-token

In [3]:
conf = spark.sparkContext._jsc.hadoopConfiguration()
conf.set("fs.gs.project.id", "hering-datalake-prod")
conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
conf.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")

---

In [4]:
from pyspark.sql.avro.functions import from_avro, to_avro
import pyspark.sql.functions as f
from urllib.request import urlopen
import json
from pyspark.sql.types import StringType
from dateutil import parser
from delta.tables import *

In [5]:
def flatten_df(nested_df):
    flat_cols = [c[0] for c in nested_df.dtypes if c[1][:6] != 'struct']
    nested_cols = [c[0] for c in nested_df.dtypes if c[1][:6] == 'struct']

    flat_df = nested_df.select(flat_cols +
                               [f.col(nc+'.'+c)
                                for nc in nested_cols
                                for c in nested_df.select(nc+'.*').columns])
    return flat_df

In [6]:
topic = 'bnsvher602.linx_db.dbo.LOJA_VENDA'
landing_path = 'gs://hering-datalake-prod-southamerica-east1-bucket-landing-zone/ecommerce/bnsvher602/linx_db/dbo/loja_vendas/'
bronze_path = 'gs://hering-datalake-prod-southamerica-east1-bucket-bronze-zone/ecommerce/bnsvher602/db_mobile/dbo/venda'
url = 'http://schema-registry-cp-schema-registry.ingestion.svc:8081/subjects/' + topic + '-value/versions/latest'

In [7]:
df_bronze = (
    spark
    .read
    .format("delta")
    .load(bronze_path)
)

In [10]:
df = df_bronze.limit(1000).toPandas()

In [19]:
df[df.TICKET == 'P0009908']

Empty DataFrame
Columns: [offset, timestamp_kafka, year, month, day, hour, ID_FILIAL, DATA, TICKET, DATA_DIGITACAO, CANCELADO, VALOR, QTDE, VALOR_VENDA, VALOR_TROCA, VALOR_PROD_BRUTO, QTDE_VENDA, QTDE_TROCA, CPF_CNPJ_CLIENTE, CLIENTE_IDENTIFICADO, CODIGO_SITE, DESCONTO, VENDEDOR, CAIXA, IDENTIFICADO_CPF_CRM, CONVERSAO_CONTATAVEL, CONVERSAO_COMPLETA, CONVERSAO_CLIENTE_NOVO, DATA_HORA_INSERT, ORIGEM, DATA_HORA_INSERT_ERP, VALOR_FRETE, COD_FORMA_PGTO, DATA_HORA_UPDATE_ERP, DATA_HORA_CANCELAMENTO, QTDE_CANCELADA, MOTIVO_CANCELAMENTO, __op, __table, __source_ts_ms, __deleted]
Index: []

[0 rows x 41 columns]

In [20]:
df 2021-07-15

offset         timestamp_kafka  year  month  day  hour  ID_FILIAL  \
0   -37111391 2022-03-25 16:57:55.705  2022      3   25    16      11428   
1   -29619079 2022-03-25 16:04:59.625  2022      3   25    16      11024   
2   -28920797 2022-03-25 16:00:21.486  2022      3   25    16      11002   
3   -34870374 2022-03-25 16:41:19.245  2022      3   25    16      11218   
4   -37111392 2022-03-25 16:57:55.705  2022      3   25    16      11428   
..        ...                     ...   ...    ...  ...   ...        ...   
995 -36225599 2022-03-25 16:51:10.721  2022      3   25    16      11306   
996 -37111640 2022-03-25 16:57:55.721  2022      3   25    16      11428   
997 -29619245 2022-03-25 16:04:59.673  2022      3   25    16      11024   
998 -28921046 2022-03-25 16:00:21.495  2022      3   25    16      11002   
999 -34870540 2022-03-25 16:41:19.251  2022      3   25    16      11218   

      DATA    TICKET  DATA_DIGITACAO  ...  VALOR_FRETE  COD_FORMA_PGTO  \
0    18823  00004452   1626361469000  ...          0.0              01   
1    17518  00044043   1513592244000  ...          NaN            None   
2    17272  16114437   1492343077000  ...          NaN            None   
3    18209  00009886   1573322100000  ...          NaN            None   
4    18823  00004453   1626368982000  ...          0.0              ##   
..     ...       ...             ...  ...          ...             ...   
995  19072  00007474   1647875361000  ...          0.0              07   
996  18845  00004957   1628244380000  ...          0.0              ##   
997  17519  00044377   1513693659000  ...          NaN            None   
998  17317  31094730   1496224050000  ...          NaN            None   
999  18221  00010193   1574342535000  ...          NaN            None   

     DATA_HORA_UPDATE_ERP  DATA_HORA_CANCELAMENTO  QTDE_CANCELADA  \
0                     NaN                     NaN             NaN   
1                     NaN                     NaN             NaN   
2                     NaN                     NaN             NaN   
3                     NaN                     NaN             NaN   
4            1.626369e+12                     NaN             NaN   
..                    ...                     ...             ...   
995                   NaN                     NaN             NaN   
996          1.628245e+12                     NaN             NaN   
997                   NaN                     NaN             NaN   
998                   NaN                     NaN             NaN   
999                   NaN                     NaN             NaN   

     MOTIVO_CANCELAMENTO  __op  __table __source_ts_ms __deleted  
0                    NaN     r    VENDA  1648227474725     false  
1                    NaN     r    VENDA  1648224298545     false  
2                    NaN     r    VENDA  1648224020581     false  
3                    NaN     r    VENDA  1648226477048     false  
4                    NaN     r    VENDA  1648227474725     false  
..                   ...   ...      ...            ...       ...  
995                  NaN     r    VENDA  1648227070318     false  
996                  NaN     r    VENDA  1648227474817     false  
997                  NaN     r    VENDA  1648224298600     false  
998                  NaN     r    VENDA  1648224020669     false  
999                  NaN     r    VENDA  1648226477137     false  

[1000 rows x 41 columns]

In [7]:
# Get Schema from schema registry
response = urlopen(url)
data_json = json.loads(response.read())

In [ ]:
# Reading last bronze data
try:
    df_bronze = (
        spark
        .read
        .format("delta")
        .load(bronze_path)
    )

    list_result = (
        df_bronze
        .agg({"timestamp_kafka": "max"}).collect()
        .where(f.col("timestamp_kafka")>"16-03-2021 12:00AM")
    )
    
    last_date = list_result[0].asDict()['max(timestamp_kafka)']
    
except:
    last_date = parser.parse("01-01-2021 12:00AM")

In [ ]:
print(last_date)

In [10]:
# Reading landing data
df_table = (
    spark
    .read
    .format("avro")
    .load(landing_path)
    .where((f.col("year")>=last_date.year)&(f.col("month")>=last_date.month)&(f.col("day")>=last_date.day))
    .where(f.col("timestamp")>last_date)
    .withColumnRenamed("timestamp", "timestamp_kafka")
)

In [11]:
from_avro_options= {"mode":"PERMISSIVE"}
df_temp = (
    df_table
    .withColumn('fixedValue', f.expr("substring(value, 6, length(value)-5)"))
    .select(from_avro("fixedValue", data_json['schema'], from_avro_options), 'offset', 'timestamp_kafka',"year", "month", "day", "hour")
)

In [12]:
df_table_flat = flatten_df(df_temp)

In [13]:
df_table_flat.count()

40987751

In [13]:
(
    df_table_flat
    .write
    .partitionBy("year", "month", "day", "hour")
    .format("delta")
    .mode("append")
    .option("checkpointLocation", bronze_path + "checkpoint")
    .save(bronze_path)
)

In [21]:
##################################
### Verificar está funcionando ###
##################################

In [16]:
count_bronze = (
    spark
    .read
    .format("delta")
    .load(bronze_path)
    .count()
)

print (count_bronze)

261660


In [18]:
landing_path_snapshot = 'gs://hering-datalake-prod-southamerica-east1-bucket-landing-zone/ecommerce/bnsvher602/linx_db/dbo/produtos_snapshot/'

count_landing = (
    spark
    .read
    .format("avro")
    .load(landing_path)
    .count()
)

count_landing_snap = (
    spark
    .read
    .format("avro")
    .load(landing_path_snapshot)
    .count()
)


print (count_landing + count_landing_snap)

261660
